<a href="https://colab.research.google.com/github/bgalerne/IoT_data_science/blob/main/3_iot_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction to image processing: Filtering in Fourier domain and variational methods for inverse problems






Import usual libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import skimage
import skimage.color

import imageio

import scipy.ndimage


Download some images

In [ ]:
!wget "https://www.idpoisson.fr/galerne/iot_data_science/parrot.bmp"
!wget "https://www.idpoisson.fr/galerne/iot_data_science/image_sharpening.png"

# Filtering in Fourier domain

## Forward and inverse Fourier transform

**Exercise:** Given an image $u$, compute its Discrete Fourier transform $\hat u$ using ```np.fft.fftshift(np.fft.fft2(u))```.

In a 3x2 subplot, display 
 * the original image 
 * the real and imaginary parts of $\hat u$ 
 * the Fourier modulus with a proper contrast change 
 * the Fourier phase (with a cyclic colormap so that $\pi$ and $-\pi$ have the same color)
 * the recontructed image by inverse FFT.
 
For the recontructed image you may need to set to zero the imaginary part. But check that it is close to zero first.




In [ ]:

imgrgb = skimage.img_as_float(imageio.imread('parrot.bmp'))
imggray = skimage.color.rgb2gray(imgrgb)
dftimg = np.fft.fftshift(np.fft.fft2(imggray))

#TODO:


## Low pass filter

**Exercise:** Reproduce the low-pass filter demo from the slide using the Parrot image (grayscale).

We will suppose that the image height and width $M,N$ are even numbers.

Spectral domain:  $\hat{\Omega}_{M,N}$: the null-frequency $(0,0)$ is at the center of the domain:
$$
\hat{\Omega}_{M,N} = \left\{ -\frac{M}{2},\dots,\frac{M}{2}-1\right\}\times \left\{ -\frac{N}{2},\dots,\frac{N}{2}-1\right\}.
$$
To produce a disk image you should make grid images reproducing these indices.

Discuss the different details that appear as the disc radius grows.



In [ ]:
#TODO.

## Image sharpening

**(Optional) Exercise:** Reproduce the experiment for image sharpening on the image
```image_sharpening.png```. 
To do the smooth mask, apply a Gaussian convolution filter to the binary disk used above.

What value of gain feels like a sharpening ? For which gain value  this become degenerate ?


In [ ]:
imgrgb = skimage.img_as_float(imageio.imread('image_sharpening.png'))
imggray = skimage.color.rgb2gray(imgrgb)

#TODO

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
im0 = ax[0].imshow(imggray, cmap=plt.cm.gray)
im0.set_clim(0,1)
ax[0].set_title("Grayscale image")
#im1 = ax[1].imshow(sharpened, cmap=plt.cm.gray) #TODO
ax[1].set_title("Sharpened")
fig.tight_layout()
plt.show()

# Variational methods for inverse problems

In [ ]:
# Define Gaussian blur operator:
sigblur = 2.
s = int(3*sigblur)
w = 2*s+1
kernel = np.zeros(w)
for t in np.arange(w):
    kernel[t] = np.exp(-(t-s)**2/(2*sigblur**2))
kernel /= sum(kernel)
gausskernel = np.zeros((w,w))
for t1 in np.arange(w):
    for t2 in np.arange(w):
        gausskernel[t1,t2] = kernel[t1]*kernel[t2]
blur = lambda x: scipy.ndimage.convolve(x, gausskernel)

original = skimage.data.astronaut()
imggray = skimage.color.rgb2gray(original)
imggray = imggray[0:200,120:320]
M, N = imggray.shape
imgblurred = blur(imggray)

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
im0 = ax[0].imshow(imggray, cmap=plt.cm.gray)
im0.set_clim(0,1)
ax[0].set_title("Grayscale")
plt.colorbar(im0,ax=ax[0])
im1 = ax[1].imshow(imgblurred, cmap=plt.cm.gray)
im1.set_clim(0,1)
ax[1].set_title("Blurred image")
plt.colorbar(im1,ax=ax[1])
fig.tight_layout()
plt.show()



## Computation of pseudo-inverse using gradient descent
Here $H$ is the linear operator such that 
$$
H x = g_\sigma \ast x = \operatorname{blur}(x).
$$
We want to minimize the function $f:\mathbb{R}^{M\times N} \to \mathbb{R}$
$$
f(x) = \frac{1}{2}\| H x - y \|^2.
$$
We have
$$
\nabla f (x) = H^T ( Hx - y)
$$
that is Lipschitz with constant $L = 1$ (attained for constant images).


In [ ]:
y = imgblurred # blur without noise
# y = imgblurred+4./255.*np.random.randn(M, N) # blur with additive noise

def f(x):
  return 0.5*np.sum((blur(x)-y)**2)

def gradf(x):
  return blur(blur(x)-y)

L = 1 

def gradient_descent_fixed_step_size(f,gradf,x0,niter,t,verb=True):
  FXk = f(x0)
  x = x0.copy()
  for k in range(niter):
    x = x - t * gradf(x)
    fx = f(x)
    FXk = np.append(FXk,fx)
    if verb:
      print(['it ', k, 'f(x) = ', fx])
  return x, FXk

t = 1/L
niter = 200
x0 = np.zeros(y.shape)
x, FXk = gradient_descent_fixed_step_size(f,gradf,x0,niter,t)


def imshowlistgray(imgs, titles):
  fig, ax = plt.subplots(1, len(imgs), figsize = (10*len(imgs),10))
  for k, im in enumerate(imgs):
    im = ax[k].imshow(im, cmap = plt.cm.gray)
    im.set_clim(0,1)
    ax[k].set_title(titles[k])

imshowlistgray([imggray, x0, x],['original', 'init', 'output'])

**Exercice:** Run the above code with a noise-free blurred input and with a blurred input corrupted with additive Gaussian noise (comment/uncomment first line). Observe that the pseudo-inverse is sensitive to noise.

## Tichonov regularization
Here we minimize the function
$$
g(x) = f(x) + \tau \frac{1}{2} \| \nabla x \|^2 
= \frac{1}{2} \| H x - y \|^2 + \frac{\tau }{2}\| \nabla x \|^2,
$$
where
$$
\| \nabla x \|^2
= \sum_{m=0}^{M-1}\sum_{n=0}^{M-1} (x(m+1,n) - x(m,n))^2 + (x(m,n+1) - x(m,n))^2
$$
is the squared-norm of the gradient vector field (that can be computed thanks to two linear filters).

One has
$$
\nabla g(x) = \nabla f (x) - \Delta x = H^T ( Hx - y) - \tau \Delta x,
$$
where $\Delta x$ is the discrete Laplacian, that is, the convolution of $x$ with the kernel
$$
\begin{pmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0
\end{pmatrix}.
$$
Here and we can take
$$
L = 1+8\tau.
$$

**Exercise:**

1. Define the necessary functions for minimizing the functional $g$ with the gradient descent code above.
2. Check that the output is consistent in the noisy-free case.
3. In the noisy case, display the results for four different values of the regularization parameter $\tau$, from too small to too high.





In [ ]:
#TODO:


